In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib

# For interactive widgets
import ipywidgets as widgets
from IPython.display import display, clear_output


In [2]:
DATASET_FILE = 'human_vital_signs_dataset_2024.csv'  

if not os.path.isfile(DATASET_FILE):
    print(f"File '{DATASET_FILE}' does not exist in the directory.")
else:
    print(f"File '{DATASET_FILE}' found in the directory.")


File 'human_vital_signs_dataset_2024.csv' found in the directory.


In [3]:
def load_dataset():
    df = pd.read_csv(DATASET_FILE)
    df.columns = df.columns.str.strip()  # Trim whitespace from column names
    print("Dataset columns:", df.columns.tolist())  # Print the columns to verify
    print("\nFirst few rows of the dataset:")
    display(df.head())  # Display the first few rows of the dataset
    return df

# Load the dataset
df = load_dataset()


Dataset columns: ['Patient ID', 'Heart Rate', 'Respiratory Rate', 'Timestamp', 'Body Temperature', 'Oxygen Saturation', 'Systolic Blood Pressure', 'Diastolic Blood Pressure', 'Age', 'Gender', 'Weight (kg)', 'Height (m)', 'Derived_HRV', 'Derived_Pulse_Pressure', 'Derived_BMI', 'Derived_MAP', 'Risk Category']

First few rows of the dataset:


,Patient ID,Heart Rate,Respiratory Rate,Timestamp,Body Temperature,Oxygen Saturation,Systolic Blood Pressure,Diastolic Blood Pressure,Age,Gender,Weight (kg),Height (m),Derived_HRV,Derived_Pulse_Pressure,Derived_BMI,Derived_MAP,Risk Category
0,1,60,12,53:45.7,36.861707,95.702046,124,86,37,Female,91.541618,1.679351,0.121033,38,32.459031,98.666667,High Risk
1,2,63,18,52:45.7,36.511633,96.689413,126,84,77,Male,50.704921,1.992546,0.117062,42,12.771246,98.000000,High Risk
2,3,63,15,51:45.7,37.052049,98.508265,131,78,68,Female,90.316760,1.770228,0.053200,53,28.821069,95.666667,Low Risk
3,4,99,16,50:45.7,36.654747,95.011801,118,72,41,Female,96.006188,1.833629,0.064475,46,28.554611,87.333333,High Risk
4,5,69,16,49:45.7,36.975098,98.623792,138,76,25,Female,56.020006,1.866419,0.118484,62,16.081438,96.666667,High Risk


In [4]:
def train_ml_model():
    # Load the dataset
    df = load_dataset()

    required_columns = ['Heart Rate', 'Respiratory Rate', 'Body Temperature', 
                        'Oxygen Saturation', 'Systolic Blood Pressure', 'Diastolic Blood Pressure', 'Risk Category']
    for col in required_columns:
        if col not in df.columns:
            raise KeyError(f"Column '{col}' not found in dataset. Available columns: {df.columns.tolist()}")

    # Features and target
    X = df[['Heart Rate', 'Respiratory Rate', 'Body Temperature', 
             'Oxygen Saturation', 'Systolic Blood Pressure', 'Diastolic Blood Pressure']]
    y = df['Risk Category']

    # Encode target variable if it's categorical
    if y.dtype == 'object':
        y = y.map({'Low Risk': 0, 'High Risk': 1})  # Adjust mappings as per your dataset

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize and train the model
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Model accuracy: {accuracy:.2f}')

    # Save the trained model
    joblib.dump(model, 'vitals_risk_model.pkl')
    print("Model saved as 'vitals_risk_model.pkl'")

    return model

# Train the model
model = train_ml_model()


Dataset columns: ['Patient ID', 'Heart Rate', 'Respiratory Rate', 'Timestamp', 'Body Temperature', 'Oxygen Saturation', 'Systolic Blood Pressure', 'Diastolic Blood Pressure', 'Age', 'Gender', 'Weight (kg)', 'Height (m)', 'Derived_HRV', 'Derived_Pulse_Pressure', 'Derived_BMI', 'Derived_MAP', 'Risk Category']

First few rows of the dataset:


,Patient ID,Heart Rate,Respiratory Rate,Timestamp,Body Temperature,Oxygen Saturation,Systolic Blood Pressure,Diastolic Blood Pressure,Age,Gender,Weight (kg),Height (m),Derived_HRV,Derived_Pulse_Pressure,Derived_BMI,Derived_MAP,Risk Category
0,1,60,12,53:45.7,36.861707,95.702046,124,86,37,Female,91.541618,1.679351,0.121033,38,32.459031,98.666667,High Risk
1,2,63,18,52:45.7,36.511633,96.689413,126,84,77,Male,50.704921,1.992546,0.117062,42,12.771246,98.000000,High Risk
2,3,63,15,51:45.7,37.052049,98.508265,131,78,68,Female,90.316760,1.770228,0.053200,53,28.821069,95.666667,Low Risk
3,4,99,16,50:45.7,36.654747,95.011801,118,72,41,Female,96.006188,1.833629,0.064475,46,28.554611,87.333333,High Risk
4,5,69,16,49:45.7,36.975098,98.623792,138,76,25,Female,56.020006,1.866419,0.118484,62,16.081438,96.666667,High Risk


Model accuracy: 0.63
Model saved as 'vitals_risk_model.pkl'


In [5]:
def load_trained_model(model_path='vitals_risk_model.pkl'):
    if not os.path.isfile(model_path):
        raise FileNotFoundError(f"Model file '{model_path}' not found. Please train the model first.")
    model = joblib.load(model_path)
    print(f"Model loaded from '{model_path}'.")
    return model

# Load the trained model
model = load_trained_model()


Model loaded from 'vitals_risk_model.pkl'.


In [6]:
def predict_risk_with_ml(model, heart_rate, respiratory_rate, body_temperature, 
                         oxygen_saturation, systolic_blood_pressure, diastolic_blood_pressure):
    """
    Predicts the risk category using a pre-trained machine learning model.
    
    Parameters:
        heart_rate (float): Heart rate in bpm.
        respiratory_rate (float): Respiratory rate in breaths/min.
        body_temperature (float): Body temperature in °C.
        oxygen_saturation (float): Oxygen saturation in %.
        systolic_blood_pressure (float): Systolic blood pressure in mmHg.
        diastolic_blood_pressure (float): Diastolic blood pressure in mmHg.
    
    Returns:
        str: Predicted risk category.
    """
    input_data = [[heart_rate, respiratory_rate, body_temperature, 
                   oxygen_saturation, systolic_blood_pressure, diastolic_blood_pressure]]

    prediction_num = model.predict(input_data)[0]
    risk_category = "High Risk" if prediction_num == 1 else "Low Risk"
    
    return risk_category

def generate_issues_report(vital_signs, risk_category, filename='patient_issues_report.txt'):
    """
    Generates a report of the patient's vital signs and predicted risk category.
    
    Parameters:
        vital_signs (dict): Dictionary containing vital sign values.
        risk_category (str): The predicted risk category.
        filename (str): The name of the report file.
    """
    issues = {
        "Low Risk": "No immediate concerns detected. Maintain a healthy lifestyle.",
        "High Risk": "Possible health issues detected. Consider consulting a healthcare provider. Conditions may include hypertension, respiratory distress, or other cardiovascular issues."
    }

    with open(filename, 'w') as file:
        file.write("Patient Vital Signs Report\n")
        file.write("==========================\n\n")
        for key, value in vital_signs.items():
            file.write(f"{key}: {value}\n")
        file.write(f"\nPredicted Risk Category: {risk_category}\n")
        file.write("\nIssues report generated as 'patient_issues_report.txt'.\n")
    
    print(f"Issues report generated as '{filename}'.")


In [7]:
# Define input widgets
heart_rate_widget = widgets.BoundedIntText(
    value=70,
    min=30,
    max=200,
    step=1,
    description='Heart Rate (bpm):',
    disabled=False
)

respiratory_rate_widget = widgets.BoundedIntText(
    value=16,
    min=5,
    max=60,
    step=1,
    description='Respiratory Rate (breaths/min):',
    disabled=False
)

body_temperature_widget = widgets.BoundedFloatText(
    value=36.5,
    min=30.0,
    max=45.0,
    step=0.1,
    description='Body Temperature (°C):',
    disabled=False
)

oxygen_saturation_widget = widgets.BoundedIntText(
    value=98,
    min=50,
    max=100,
    step=1,
    description='Oxygen Saturation (%):',
    disabled=False
)

systolic_bp_widget = widgets.BoundedIntText(
    value=120,
    min=50,
    max=250,
    step=1,
    description='Systolic BP (mmHg):',
    disabled=False
)

diastolic_bp_widget = widgets.BoundedIntText(
    value=80,
    min=30,
    max=150,
    step=1,
    description='Diastolic BP (mmHg):',
    disabled=False
)

submit_button = widgets.Button(
    description='Submit',
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to predict risk',
    icon='check'
)

output = widgets.Output()

# Display the widgets
display(heart_rate_widget,
        respiratory_rate_widget,
        body_temperature_widget,
        oxygen_saturation_widget,
        systolic_bp_widget,
        diastolic_bp_widget,
        submit_button,
        output)


BoundedIntText(value=70, description='Heart Rate (bpm):', max=200, min=30)

BoundedIntText(value=16, description='Respiratory Rate (breaths/min):', max=60, min=5)

BoundedFloatText(value=36.5, description='Body Temperature (°C):', max=45.0, min=30.0, step=0.1)

BoundedIntText(value=98, description='Oxygen Saturation (%):', min=50)

BoundedIntText(value=120, description='Systolic BP (mmHg):', max=250, min=50)

BoundedIntText(value=80, description='Diastolic BP (mmHg):', max=150, min=30)

Button(button_style='success', description='Submit', icon='check', style=ButtonStyle(), tooltip='Click to pred…

Output()

In [8]:
def on_submit_button_clicked(b):
    with output:
        clear_output()
        # Retrieve values from widgets
        heart_rate = heart_rate_widget.value
        respiratory_rate = respiratory_rate_widget.value
        body_temperature = body_temperature_widget.value
        oxygen_saturation = oxygen_saturation_widget.value
        systolic_bp = systolic_bp_widget.value
        diastolic_bp = diastolic_bp_widget.value
        
        # Prepare vital signs data
        vital_signs = {
            "Heart Rate (bpm)": heart_rate,
            "Respiratory Rate (breaths/min)": respiratory_rate,
            "Body Temperature (°C)": body_temperature,
            "Oxygen Saturation (%)": oxygen_saturation,
            "Systolic Blood Pressure (mmHg)": systolic_bp,
            "Diastolic Blood Pressure (mmHg)": diastolic_bp
        }
        
        # Predict risk category using the ML model
        risk_category = predict_risk_with_ml(
            model, 
            heart_rate, 
            respiratory_rate, 
            body_temperature, 
            oxygen_saturation, 
            systolic_bp, 
            diastolic_bp
        )
        
        # Display the prediction
        print(f"Predicted Risk Category: {risk_category}")
        
        # Generate the report
        generate_issues_report(vital_signs, risk_category)

# Bind the function to the button
submit_button.on_click(on_submit_button_clicked)
